In [1]:
# Install BigQuery client library (run this cell once)
%pip install google-cloud-bigquery --quiet
%pip install bigquery-magics --quiet
%pip install db-dtypes --quiet
# Import libraries
from google.cloud import bigquery
import pandas as pd 
%load_ext bigquery_magics
import db_dtypes
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "salesforce-465614-2cf9e37da64b.json"
from google.cloud import bigquery
# Initialize BigQuery client
client = bigquery.Client()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
%%bqsql
 select * from salesforce-465614.cust_analytics.report_transactions limit 2

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_id,customer_id,product_id,Store_id,order_status,Purchase_Date,category,sub_category,Unit_Price,quantity,Total_Price,payment_method,channel,loyalty_program,age,gender,city
0,T98509,C1330,P419,STORE_7,Completed,2024-03-07,Electronics,Accessories,113.77,3,341.31,Bank Transfer,Physical,true,20,Female,Port Jeromeshire
1,T517643,C1952,P805,STORE_4,Cancelled,2024-03-07,Toys,Board Games,207.78,2,415.56,Bank Transfer,Physical,false,66,Male,Williamsland


In [3]:
%%bqsql
SELECT 
  category,
  channel,
  SUM(total_price) AS total_revenue,
  COUNT(*) AS transaction_count
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY category, channel
ORDER BY total_revenue DESC
limit 5


Query is running:   0%|          |

Downloading:   0%|          |

,category,channel,total_revenue,transaction_count
0,Books,Physical,5651307.75,1972
1,Beauty,Physical,4672627.00,1706
2,Electronics,Physical,4256474.44,1405
3,Clothing,Physical,4091927.72,1632
4,Toys,Physical,3193546.68,1180


In [4]:
%%bqsql
    SELECT 
  gender,
  loyalty_program,
  CASE 
    WHEN age < 25 THEN 'Under 25'
    WHEN age BETWEEN 25 AND 34 THEN '25-34'
    WHEN age BETWEEN 35 AND 44 THEN '35-44'
    WHEN age BETWEEN 45 AND 54 THEN '45-54'
    ELSE '55+' 
  END AS age_group,
  COUNT(*) AS purchase_count,
  SUM(total_price) AS total_spent
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY gender, loyalty_program, age_group
ORDER BY total_spent DESC
limit 10


Query is running:   0%|          |

Downloading:   0%|          |

,gender,loyalty_program,age_group,purchase_count,total_spent
0,Male,true,55+,2367,6587862.31
1,Female,true,55+,1655,4594675.71
2,Male,false,55+,1019,2777286.20
3,Male,true,35-44,908,2542308.96
4,Male,true,25-34,919,2506794.31
5,Male,true,45-54,854,2391709.80
6,Female,true,45-54,630,1754027.33
7,Female,false,55+,605,1735068.99
8,Male,true,Under 25,630,1705714.24
9,Female,true,35-44,577,1658710.10


In [5]:
%%bqsql
 SELECT 
  channel,
  COUNT(*) AS total_transactions,
  round(AVG(total_price),2) AS avg_order_value,
  SUM(total_price) AS total_sales
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY channel;

Query is running:   0%|          |

Downloading:   0%|          |

,channel,total_transactions,avg_order_value,total_sales
0,Online,4478,2763.17,12373485.93
1,Physical,8954,2791.26,24992961.94


In [6]:
%%bqsql
SELECT 
  order_status,
  COUNT(*) AS order_count,
  SUM(total_price) AS total_value
FROM `salesforce-465614.cust_analytics.report_transactions`
GROUP BY order_status
ORDER BY order_count DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,order_status,order_count,total_value
0,Completed,13432,37366447.87
1,Cancelled,6568,18545186.49


In [9]:
%%bqsql
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2   limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days
0,C1775,1
1,C1656,1
2,C1257,1
3,C815,1
4,C1507,1
5,C1482,1
6,C222,1
7,C1735,1
8,C256,1
9,C1670,1


In [11]:
%%bqsql
SELECT 
  customer_id,
  COUNT(*) AS frequency,
  SUM(total_price) AS total_spent
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2 desc limit 10;


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,frequency,total_spent
0,C651,15,34439.83
1,C1232,15,36490.19
2,C1716,15,26711.96
3,C1226,15,52173.05
4,C315,15,57776.33
5,C417,14,33705.18
6,C1651,14,34717.01
7,C1670,14,39171.39
8,C1270,14,52783.92
9,C422,14,42538.85


In [12]:
%%bqsql
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,
  COUNT(*) AS frequency,
  SUM(total_price) AS total_spent
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id
order by 2, 3 desc
limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,frequency,total_spent
0,C1670,1,14,39171.39
1,C1482,1,11,27134.22
2,C1790,1,10,29725.95
3,C1859,1,10,31524.75
4,C960,1,10,18172.30
5,C860,1,10,35283.59
6,C1257,1,10,35389.16
7,C1735,1,9,18960.75
8,C866,1,9,25451.32
9,C333,1,9,23258.66


In [13]:
%%bqsql
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,
  CASE 
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY)  <= 30 THEN 'Recent'
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 90 THEN 'Moderate'
    ELSE 'Stale'
  END AS recency_bucket,
  COUNT(*) AS frequency,
  CASE 
    WHEN COUNT(*) >= 10 THEN 'Frequent'
    WHEN COUNT(*) >= 5 THEN 'Moderate'
    ELSE 'Less Frequent'
  END AS frequency_bucket,
  SUM(total_price) AS total_spent,
  CASE 
    WHEN SUM(total_price) >= 10000 THEN 'High Spender'
    WHEN SUM(total_price) >= 5000 THEN 'Medium Spender'
    ELSE 'Low Spender'
  END AS Spending_bucket
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id;


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,recency_bucket,frequency,frequency_bucket,total_spent,Spending_bucket
0,C691,5,Recent,7,Moderate,16639.07,High Spender
1,C352,156,Stale,9,Moderate,28521.41,High Spender
2,C756,23,Recent,6,Moderate,9307.33,Medium Spender
3,C730,22,Recent,7,Moderate,24009.38,High Spender
4,C1816,32,Moderate,6,Moderate,9098.31,Medium Spender
...,...,...,...,...,...,...,...
1992,C46,18,Recent,5,Moderate,11305.74,High Spender
1993,C1695,17,Recent,3,Less Frequent,8758.04,Medium Spender
1994,C6,90,Moderate,7,Moderate,15080.18,High Spender
1995,C774,153,Stale,3,Less Frequent,6158.07,Medium Spender


In [31]:
%%bqsql
create or replace view `salesforce-465614.cust_analytics.customer_segments` as
SELECT 
  customer_id,
  DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) AS recency_days,
  COUNT(*) AS frequency,
  SUM(total_price) AS total_spent,
  CASE 
     WHEN
    DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 THEN 'Recent'
    when DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 90 THEN 'Moderate'
    ELSE 'Stale'
  END AS recency_bucket,
     CASE 
     WHEN COUNT(*) >= 10 THEN 'Frequent'
     WHEN COUNT(*) >= 5 THEN 'Moderate'
     ELSE 'Less Frequent'
     END AS frequency_bucket,
  CASE 
    WHEN SUM(total_price) >= 10000 THEN 'High Spender'
    WHEN SUM(total_price) >= 5000 THEN 'Medium Spender'
    ELSE 'Low Spender'
  END AS spending_bucket,
  CASE
    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) >= 10 
         AND SUM(total_price) >= 10000 THEN 'Loyal Customer'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) BETWEEN 5 AND 9 
         AND SUM(total_price) >= 5000 THEN 'Potential Customer'

     WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) BETWEEN 31 AND 90 
         AND COUNT(*) between 5 and 9
         AND SUM(total_price) > 5000 THEN 'Mindful Spender'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) BETWEEN 31 AND 90 
         AND COUNT(*) <= 4 
         AND SUM(total_price) < 5000 THEN 'At Risk'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) > 90 
         AND COUNT(*) < 3 THEN 'Churned'

    WHEN COUNT(*) >= 10 
         AND SUM(total_price) >= 10000 THEN 'Big Spender'

    WHEN DATE_DIFF(date('2024-09-24'), MAX(purchase_date), DAY) <= 30 
         AND COUNT(*) < 3 THEN 'New Customer'
    ELSE 'Other'
  END AS customer_segment
FROM `salesforce-465614.cust_analytics.report_transactions`
WHERE order_status = 'Completed'
GROUP BY customer_id;
select * from `salesforce-465614.cust_analytics.customer_segments` limit 10


Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,recency_days,frequency,total_spent,recency_bucket,frequency_bucket,spending_bucket,customer_segment
0,C585,79,10,31489.20,Moderate,Frequent,High Spender,Big Spender
1,C1683,12,8,26250.93,Recent,Moderate,High Spender,Potential Customer
2,C1529,10,10,36022.93,Recent,Frequent,High Spender,Loyal Customer
3,C1162,26,8,32605.42,Recent,Moderate,High Spender,Potential Customer
4,C1354,13,9,27375.15,Recent,Moderate,High Spender,Potential Customer
5,C730,22,7,24009.38,Recent,Moderate,High Spender,Potential Customer
6,C106,23,8,19503.59,Recent,Moderate,High Spender,Potential Customer
7,C1477,27,3,12250.05,Recent,Less Frequent,High Spender,Other
8,C1739,117,5,15886.50,Stale,Moderate,High Spender,Other
9,C467,10,7,19100.46,Recent,Moderate,High Spender,Potential Customer


In [18]:
%%bqsql
select rpt_trans.*,cust_seg.* except (customer_id) from salesforce-465614.cust_analytics.report_transactions as rpt_trans
left join salesforce-465614.cust_analytics.customer_segments as cust_seg
on rpt_trans.customer_id = cust_seg.customer_id
where rpt_trans.order_status = 'Completed' limit 3

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_id,customer_id,product_id,Store_id,order_status,Purchase_Date,category,sub_category,Unit_Price,quantity,...,age,gender,city,recency_days,frequency,total_spent,recency_bucket,frequency_bucket,spending_bucket,customer_segment
0,T528336,C1342,P3,STORE_29,Completed,2024-03-02,Beauty,Makeup,529.42,7,...,57,Male,Melissaton,41,8,23297.54,Moderate,Moderate,High Spender,Other
1,T417425,C1421,P845,STORE_33,Completed,2024-03-02,Beauty,Haircare,927.10,5,...,38,Male,Gailtown,1,5,21587.38,Recent,Moderate,High Spender,Potential Customer
2,T133200,C1699,P215,STORE_14,Completed,2024-03-02,Beauty,Makeup,879.28,5,...,67,Male,Connietown,64,10,25832.75,Moderate,Frequent,High Spender,Big Spender


In [41]:
%%bqsql
select loyalty_program,sum(total_spent),(sum(total_spent)/sum(sum(total_spent)) over())*100 from (
select rpt_trans.*,cust_seg.* except (customer_id) from salesforce-465614.cust_analytics.report_transactions as rpt_trans
left join salesforce-465614.cust_analytics.customer_segments as cust_seg
on rpt_trans.customer_id = cust_seg.customer_id
where rpt_trans.order_status = 'Completed' )
group by loyalty_program order by 2 desc

Query is running:   0%|          |

Downloading:   0%|          |

,loyalty_program,f0_,f1_
0,true,2.038456e+08,70.94943
1,false,8.346552e+07,29.05057
